# 🚀 Advanced Hyperparameter Optimization System

## Enhanced optimization framework with:
- **Study Resumption**: Load and continue existing optimizations
- **Multi-Symbol Optimization**: Optimize across all 7 currency pairs
- **Parameter Transfer**: Apply successful parameters across symbols
- **Benchmarking Dashboard**: Compare optimization performance
- **Ensemble Methods**: Combine multiple best models
- **Adaptive Systems**: Market regime detection and switching

Built on existing optimization results from previous runs.

In [6]:
# Advanced Hyperparameter Optimization Framework
import os
import sys
import json
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any
import logging
from dataclasses import dataclass
from collections import defaultdict

warnings.filterwarnings('ignore')

# Setup enhanced logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('advanced_optimization.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Import optimization libraries
try:
    import optuna
    from optuna.samplers import TPESampler, CmaEsSampler
    from optuna.pruners import MedianPruner, HyperbandPruner
    from optuna.study import MaxTrialsCallback
    from optuna.trial import TrialState
    print("✅ Optuna available")
except ImportError:
    print("Installing Optuna with advanced features...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "optuna[optional]"])
    import optuna
    from optuna.samplers import TPESampler, CmaEsSampler
    from optuna.pruners import MedianPruner, HyperbandPruner
    print("✅ Optuna installed with advanced features")

# ML and deep learning imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam, RMSprop, SGD

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold, RFE

# Technical analysis
import ta
from ta.volatility import BollingerBands, AverageTrueRange
from ta.trend import ADXIndicator, MACD, CCIIndicator
from ta.momentum import StochasticOscillator, ROCIndicator, RSIIndicator

# Configuration
SYMBOLS = ['EURUSD', 'GBPUSD', 'USDJPY', 'AUDUSD', 'USDCAD', 'EURJPY', 'GBPJPY']
DATA_PATH = "data"
RESULTS_PATH = "optimization_results"
MODELS_PATH = "exported_models"

# Advanced optimization settings
ADVANCED_CONFIG = {
    'n_trials_per_symbol': 50,
    'n_trials_ensemble': 25,
    'cv_splits': 5,
    'timeout_per_symbol': 1800,  # 30 minutes per symbol
    'n_jobs': 1,  # Sequential for stability
    'study_storage': None,  # In-memory for now
    'enable_pruning': True,
    'enable_warm_start': True,
    'enable_transfer_learning': True
}

print(f"🎯 Advanced Optimization System Initialized")
print(f"Target symbols: {SYMBOLS}")
print(f"Configuration: {ADVANCED_CONFIG}")

# Suppress TensorFlow warnings
tf.get_logger().setLevel('ERROR')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

✅ Optuna available
🎯 Advanced Optimization System Initialized
Target symbols: ['EURUSD', 'GBPUSD', 'USDJPY', 'AUDUSD', 'USDCAD', 'EURJPY', 'GBPJPY']
Configuration: {'n_trials_per_symbol': 50, 'n_trials_ensemble': 25, 'cv_splits': 5, 'timeout_per_symbol': 1800, 'n_jobs': 1, 'study_storage': None, 'enable_pruning': True, 'enable_warm_start': True, 'enable_transfer_learning': True}


In [7]:
# Data Classes for Optimization Results
@dataclass
class OptimizationResult:
    """Data class to store optimization results"""
    symbol: str
    timestamp: str
    objective_value: float
    best_params: Dict[str, Any]
    mean_accuracy: float
    mean_sharpe: float
    std_accuracy: float
    std_sharpe: float
    num_features: int
    total_trials: int
    completed_trials: int
    study_name: str
    
@dataclass
class BenchmarkMetrics:
    """Benchmark comparison metrics"""
    symbol: str
    current_score: float
    previous_best: float
    improvement: float
    rank: int
    percentile: float

print("✅ Data classes defined successfully")

✅ Data classes defined successfully


In [8]:
class AdvancedOptimizationManager:
    """Main class for managing advanced hyperparameter optimization"""
    
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        self.results_path = Path(RESULTS_PATH)
        self.models_path = Path(MODELS_PATH)
        self.results_path.mkdir(exist_ok=True)
        self.models_path.mkdir(exist_ok=True)
        
        # Initialize storage for results
        self.optimization_history: Dict[str, List[OptimizationResult]] = defaultdict(list)
        self.benchmark_results: Dict[str, BenchmarkMetrics] = {}
        self.best_parameters: Dict[str, Dict[str, Any]] = {}
        
        # Load existing results
        self.load_existing_results()
        
        logger.info(f"AdvancedOptimizationManager initialized with {len(self.optimization_history)} symbols")
    
    def load_existing_results(self):
        """Load all existing optimization results for benchmarking"""
        print("📊 Loading existing optimization results...")
        
        # Load best parameters files
        param_files = list(self.results_path.glob("best_params_*.json"))
        
        for param_file in param_files:
            try:
                with open(param_file, 'r') as f:
                    data = json.load(f)
                    
                symbol = data.get('symbol', 'UNKNOWN')
                timestamp = data.get('timestamp', 'UNKNOWN')
                
                result = OptimizationResult(
                    symbol=symbol,
                    timestamp=timestamp,
                    objective_value=data.get('objective_value', 0.0),
                    best_params=data.get('best_params', {}),
                    mean_accuracy=data.get('mean_accuracy', 0.0),
                    mean_sharpe=data.get('mean_sharpe', 0.0),
                    std_accuracy=data.get('std_accuracy', 0.0),
                    std_sharpe=data.get('std_sharpe', 0.0),
                    num_features=data.get('num_features', 0),
                    total_trials=data.get('total_trials', 0),
                    completed_trials=data.get('completed_trials', 0),
                    study_name=f"{symbol}_{timestamp}"
                )
                
                self.optimization_history[symbol].append(result)
                
                # Keep track of best parameters per symbol
                if symbol not in self.best_parameters or result.objective_value > self.best_parameters[symbol].get('objective_value', 0):
                    self.best_parameters[symbol] = {
                        'objective_value': result.objective_value,
                        'params': result.best_params,
                        'timestamp': timestamp
                    }
                
                print(f"  ✅ Loaded {symbol} optimization from {timestamp}: {result.objective_value:.4f}")
                
            except Exception as e:
                logger.warning(f"Failed to load {param_file}: {e}")
        
        print(f"\n📈 Historical Results Summary:")
        for symbol in SYMBOLS:
            if symbol in self.optimization_history:
                results = self.optimization_history[symbol]
                best_score = max(r.objective_value for r in results)
                print(f"  {symbol}: {len(results)} runs, best score: {best_score:.4f}")
            else:
                print(f"  {symbol}: No historical data")
    
    def get_warm_start_params(self, symbol: str) -> Optional[Dict[str, Any]]:
        """Get best known parameters for warm starting optimization"""
        if symbol in self.best_parameters:
            return self.best_parameters[symbol]['params']
        
        # If no specific symbol data, try to use EURUSD as baseline
        if 'EURUSD' in self.best_parameters and symbol != 'EURUSD':
            logger.info(f"Using EURUSD parameters as warm start for {symbol}")
            return self.best_parameters['EURUSD']['params']
        
        return None
    
    def calculate_benchmark_metrics(self, symbol: str, current_score: float) -> BenchmarkMetrics:
        """Calculate benchmark metrics for a new optimization result"""
        if symbol not in self.optimization_history:
            return BenchmarkMetrics(
                symbol=symbol,
                current_score=current_score,
                previous_best=0.0,
                improvement=current_score,
                rank=1,
                percentile=100.0
            )
        
        historical_scores = [r.objective_value for r in self.optimization_history[symbol]]
        previous_best = max(historical_scores)
        improvement = current_score - previous_best
        
        # Calculate rank and percentile
        all_scores = historical_scores + [current_score]
        all_scores.sort(reverse=True)
        rank = all_scores.index(current_score) + 1
        percentile = (len(all_scores) - rank + 1) / len(all_scores) * 100
        
        return BenchmarkMetrics(
            symbol=symbol,
            current_score=current_score,
            previous_best=previous_best,
            improvement=improvement,
            rank=rank,
            percentile=percentile
        )

class StudyManager:
    """Manager for Optuna studies with resumption and warm start capabilities"""
    
    def __init__(self, opt_manager: AdvancedOptimizationManager):
        self.opt_manager = opt_manager
        self.studies: Dict[str, optuna.Study] = {}
        self.study_configs: Dict[str, Dict[str, Any]] = {}
    
    def load_existing_study(self, symbol: str, study_file: str) -> Optional[optuna.Study]:
        """Load an existing study from pickle file"""
        study_path = Path(RESULTS_PATH) / study_file
        
        if not study_path.exists():
            logger.warning(f"Study file not found: {study_path}")
            return None
        
        try:
            with open(study_path, 'rb') as f:
                study = pickle.load(f)
                logger.info(f"Loaded existing study for {symbol}: {len(study.trials)} trials")
                return study
        except Exception as e:
            logger.error(f"Failed to load study {study_path}: {e}")
            return None
    
    def create_or_resume_study(
        self, 
        symbol: str, 
        sampler_type: str = 'tpe',
        pruner_type: str = 'median',
        resume_if_exists: bool = True
    ) -> optuna.Study:
        """Create a new study or resume existing one"""
        
        study_name = f"advanced_cnn_lstm_{symbol}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        
        # Try to find and resume existing study
        if resume_if_exists:
            existing_studies = list(Path(RESULTS_PATH).glob(f"study_{symbol}_*.pkl"))
            if existing_studies:
                # Get the most recent study
                latest_study_file = max(existing_studies, key=lambda x: x.stat().st_mtime)
                study = self.load_existing_study(symbol, latest_study_file.name)
                if study is not None:
                    logger.info(f"Resuming existing study for {symbol}")
                    self.studies[symbol] = study
                    return study
        
        # Create new study with specified sampler and pruner
        logger.info(f"Creating new study for {symbol}")
        
        # Configure sampler
        if sampler_type == 'tpe':
            sampler = TPESampler(
                seed=42,
                n_startup_trials=10,
                n_ei_candidates=24
            )
        elif sampler_type == 'cmaes':
            sampler = CmaEsSampler(seed=42)
        else:
            sampler = TPESampler(seed=42)
        
        # Configure pruner
        if pruner_type == 'median':
            pruner = MedianPruner(
                n_startup_trials=5,
                n_warmup_steps=10,
                interval_steps=1
            )
        elif pruner_type == 'hyperband':
            pruner = HyperbandPruner(
                min_resource=10,
                max_resource=100,
                reduction_factor=3
            )
        else:
            pruner = MedianPruner()
        
        # Create study
        study = optuna.create_study(
            direction='maximize',
            sampler=sampler,
            pruner=pruner,
            study_name=study_name
        )
        
        # Add warm start trials if available
        self.add_warm_start_trials(study, symbol)
        
        self.studies[symbol] = study
        self.study_configs[symbol] = {
            'sampler_type': sampler_type,
            'pruner_type': pruner_type,
            'study_name': study_name
        }
        
        return study
    
    def add_warm_start_trials(self, study: optuna.Study, symbol: str, max_warm_trials: int = 5):
        """Add warm start trials from best known parameters"""
        warm_params = self.opt_manager.get_warm_start_params(symbol)
        
        if warm_params is None:
            logger.info(f"No warm start parameters available for {symbol}")
            return
        
        logger.info(f"Adding warm start trials for {symbol}")
        
        # Add the exact best parameters
        try:
            study.enqueue_trial(warm_params)
            logger.info(f"Enqueued exact best parameters for {symbol}")
        except Exception as e:
            logger.warning(f"Failed to enqueue exact parameters: {e}")
        
        # Add variations of the best parameters
        for i in range(max_warm_trials - 1):
            try:
                varied_params = self.create_parameter_variation(warm_params, variation_factor=0.1 + i * 0.05)
                study.enqueue_trial(varied_params)
                logger.info(f"Enqueued variation {i+1} for {symbol}")
            except Exception as e:
                logger.warning(f"Failed to enqueue variation {i+1}: {e}")
    
    def create_parameter_variation(self, base_params: Dict[str, Any], variation_factor: float = 0.1) -> Dict[str, Any]:
        """Create a variation of base parameters for warm start"""
        varied_params = base_params.copy()
        
        # Vary numerical parameters
        numerical_params = [
            'conv1d_filters_1', 'conv1d_filters_2', 'lstm_units', 'dense_units',
            'dropout_rate', 'learning_rate', 'l1_reg', 'l2_reg',
            'confidence_threshold_high', 'confidence_threshold_low'
        ]
        
        for param in numerical_params:
            if param in varied_params:
                original_value = varied_params[param]
                if isinstance(original_value, (int, float)):
                    # Add random variation
                    if param in ['conv1d_filters_1', 'conv1d_filters_2', 'lstm_units', 'dense_units']:
                        # Integer parameters - vary by ±20%
                        variation = int(original_value * variation_factor * np.random.uniform(-1, 1))
                        varied_params[param] = max(1, original_value + variation)
                    else:
                        # Float parameters - vary by ±variation_factor
                        variation = original_value * variation_factor * np.random.uniform(-1, 1)
                        varied_params[param] = max(0.001, original_value + variation)
        
        return varied_params
    
    def save_study(self, symbol: str, study: optuna.Study):
        """Save study to pickle file"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        study_file = Path(RESULTS_PATH) / f"study_{symbol}_{timestamp}.pkl"
        
        try:
            with open(study_file, 'wb') as f:
                pickle.dump(study, f)
            logger.info(f"Study saved: {study_file}")
        except Exception as e:
            logger.error(f"Failed to save study: {e}")
    
    def get_study_summary(self, symbol: str) -> Dict[str, Any]:
        """Get summary of study progress"""
        if symbol not in self.studies:
            return {}
        
        study = self.studies[symbol]
        
        return {
            'symbol': symbol,
            'total_trials': len(study.trials),
            'completed_trials': len([t for t in study.trials if t.state == TrialState.COMPLETE]),
            'pruned_trials': len([t for t in study.trials if t.state == TrialState.PRUNED]),
            'failed_trials': len([t for t in study.trials if t.state == TrialState.FAIL]),
            'best_value': study.best_value if study.best_trial else None,
            'best_params': study.best_params if study.best_trial else None,
            'study_name': self.study_configs.get(symbol, {}).get('study_name', 'unknown')
        }

# Initialize the optimization manager and study manager
opt_manager = AdvancedOptimizationManager(ADVANCED_CONFIG)
study_manager = StudyManager(opt_manager)

print(f"\n✅ Optimization Management System Ready!")
print(f"Loaded historical data for {len(opt_manager.optimization_history)} symbols")
print(f"Best parameters available for: {list(opt_manager.best_parameters.keys())}")
print("✅ Study Manager initialized with resumption and warm start capabilities")

2025-06-13 01:06:58,582 - __main__ - WARNING - Failed to load optimization_results/best_params_EURUSD_20250612_101803.json: Expecting value: line 1 column 1 (char 0)
2025-06-13 01:06:58,586 - __main__ - WARNING - Failed to load optimization_results/best_params_EURUSD_20250612_102853.json: Expecting value: line 1 column 1 (char 0)
2025-06-13 01:06:58,590 - __main__ - WARNING - Failed to load optimization_results/best_params_EURUSD_20250612_123846.json: Expecting value: line 1 column 1 (char 0)
2025-06-13 01:06:58,593 - __main__ - WARNING - Failed to load optimization_results/best_params_EURUSD_20250612_162909.json: Expecting value: line 1 column 1 (char 0)
2025-06-13 01:06:58,597 - __main__ - WARNING - Failed to load optimization_results/best_params_EURUSD_20250612_163647.json: Expecting value: line 1 column 1 (char 0)
2025-06-13 01:06:58,600 - __main__ - WARNING - Failed to load optimization_results/best_params_EURUSD_20250612_163830.json: Expecting value: line 1 column 1 (char 0)
2025

📊 Loading existing optimization results...
  ✅ Loaded EURUSD optimization from 20250612_165248: 0.5142
  ✅ Loaded EURUSD optimization from 20250612_201934: 0.5746
  ✅ Loaded EURUSD optimization from 20250612_224109: 0.8922
  ✅ Loaded EURUSD optimization from 20250612_224206: 0.6990
  ✅ Loaded EURUSD optimization from 20250612_224209: 0.7834
  ✅ Loaded EURUSD optimization from 20250612_224322: 0.7860
  ✅ Loaded EURUSD optimization from 20250612_225026: 0.8906
  ✅ Loaded EURUSD optimization from 20250613_001206: 0.9448
  ✅ Loaded EURUSD optimization from 20250613_003126: 0.8990
  ✅ Loaded GBPUSD optimization from 20250612_224212: 0.7494
  ✅ Loaded USDJPY optimization from 20250612_224215: 0.7752

📈 Historical Results Summary:
  EURUSD: 9 runs, best score: 0.9448
  GBPUSD: 1 runs, best score: 0.7494
  USDJPY: 1 runs, best score: 0.7752
  AUDUSD: No historical data
  USDCAD: No historical data
  EURJPY: No historical data
  GBPJPY: No historical data

✅ Optimization Management System Ready

## 2. Study Resumption and Warm Start System

In [9]:
class MultiSymbolOptimizer:
    """Advanced multi-symbol hyperparameter optimizer"""
    
    def __init__(
        self, 
        opt_manager: AdvancedOptimizationManager,
        study_manager: StudyManager
    ):
        self.opt_manager = opt_manager
        self.study_manager = study_manager
        self.optimization_results: Dict[str, OptimizationResult] = {}
        
    def suggest_advanced_hyperparameters(self, trial: optuna.Trial, symbol: str = None) -> Dict[str, Any]:
        """Enhanced hyperparameter space with symbol-specific adjustments"""
        
        # Base parameter space
        params = {
            # Data parameters
            'lookback_window': trial.suggest_int('lookback_window', 10, 60),
            'feature_selection_method': trial.suggest_categorical(
                'feature_selection_method', 
                ['all', 'top_correlation', 'variance_threshold', 'rfe', 'mutual_info']
            ),
            'max_features': trial.suggest_int('max_features', 15, 40),
            'scaler_type': trial.suggest_categorical('scaler_type', ['standard', 'robust', 'minmax']),
            
            # Model architecture
            'conv1d_filters_1': trial.suggest_int('conv1d_filters_1', 32, 256, step=16),
            'conv1d_filters_2': trial.suggest_int('conv1d_filters_2', 16, 128, step=8),
            'conv1d_kernel_size': trial.suggest_int('conv1d_kernel_size', 2, 7),
            'lstm_units': trial.suggest_int('lstm_units', 25, 150, step=5),
            'lstm_return_sequences': trial.suggest_categorical('lstm_return_sequences', [True, False]),
            'dense_units': trial.suggest_int('dense_units', 10, 100, step=5),
            'num_dense_layers': trial.suggest_int('num_dense_layers', 1, 3),
            
            # Regularization
            'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.6),
            'l1_reg': trial.suggest_float('l1_reg', 1e-6, 1e-2, log=True),
            'l2_reg': trial.suggest_float('l2_reg', 1e-6, 1e-2, log=True),
            'batch_normalization': trial.suggest_categorical('batch_normalization', [True, False]),
            
            # Training parameters
            'optimizer': trial.suggest_categorical('optimizer', ['adam', 'rmsprop', 'sgd']),
            'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True),
            'batch_size': trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256]),
            'epochs': trial.suggest_int('epochs', 50, 300),
            'patience': trial.suggest_int('patience', 5, 25),
            'reduce_lr_patience': trial.suggest_int('reduce_lr_patience', 3, 15),
            
            # Trading parameters
            'confidence_threshold_high': trial.suggest_float('confidence_threshold_high', 0.55, 0.85),
            'confidence_threshold_low': trial.suggest_float('confidence_threshold_low', 0.15, 0.45),
            'signal_smoothing': trial.suggest_categorical('signal_smoothing', [True, False]),
            
            # Advanced features
            'use_rcs_features': trial.suggest_categorical('use_rcs_features', [True, False]),
            'use_cross_pair_features': trial.suggest_categorical('use_cross_pair_features', [True, False]),
        }
        
        # Ensure threshold consistency
        if params['confidence_threshold_low'] >= params['confidence_threshold_high']:
            params['confidence_threshold_low'] = params['confidence_threshold_high'] - 0.1
        
        # Symbol-specific adjustments
        if symbol:
            if symbol in ['USDJPY', 'EURJPY', 'GBPJPY']:  # JPY pairs might need different thresholds
                # JPY pairs often have different volatility characteristics
                params['confidence_threshold_high'] = trial.suggest_float('confidence_threshold_high_jpy', 0.60, 0.90)
                params['confidence_threshold_low'] = trial.suggest_float('confidence_threshold_low_jpy', 0.10, 0.40)
        
        return params
    
    def create_advanced_model(self, input_shape: Tuple[int, int], params: Dict[str, Any]) -> tf.keras.Model:
        """Create advanced model with enhanced architecture"""
        
        model = Sequential()
        
        # First Conv1D layer
        model.add(Conv1D(
            filters=params['conv1d_filters_1'],
            kernel_size=params['conv1d_kernel_size'],
            activation='relu',
            input_shape=input_shape,
            kernel_regularizer=l1_l2(l1=params['l1_reg'], l2=params['l2_reg'])
        ))
        
        if params.get('batch_normalization', True):
            model.add(BatchNormalization())
        
        model.add(Dropout(params['dropout_rate']))
        
        # Second Conv1D layer
        model.add(Conv1D(
            filters=params['conv1d_filters_2'],
            kernel_size=params['conv1d_kernel_size'],
            activation='relu',
            kernel_regularizer=l1_l2(l1=params['l1_reg'], l2=params['l2_reg'])
        ))
        
        if params.get('batch_normalization', True):
            model.add(BatchNormalization())
        
        model.add(Dropout(params['dropout_rate']))
        
        # LSTM layer(s)
        return_sequences = params.get('lstm_return_sequences', False)
        model.add(LSTM(
            units=params['lstm_units'],
            return_sequences=return_sequences,
            kernel_regularizer=l1_l2(l1=params['l1_reg'], l2=params['l2_reg']),
            recurrent_regularizer=l1_l2(l1=params['l1_reg'], l2=params['l2_reg'])
        ))
        
        if return_sequences:
            # Add another LSTM layer if return_sequences=True
            model.add(LSTM(
                units=params['lstm_units'] // 2,
                kernel_regularizer=l1_l2(l1=params['l1_reg'], l2=params['l2_reg'])
            ))
        
        if params.get('batch_normalization', True):
            model.add(BatchNormalization())
        
        model.add(Dropout(params['dropout_rate']))
        
        # Dense layers
        num_dense_layers = params.get('num_dense_layers', 1)
        for i in range(num_dense_layers):
            units = params['dense_units'] // (i + 1)
            if units < 5:
                break
            
            model.add(Dense(
                units=units,
                activation='relu',
                kernel_regularizer=l1_l2(l1=params['l1_reg'], l2=params['l2_reg'])
            ))
            
            if i < num_dense_layers - 1:  # Don't add dropout before final layer
                model.add(Dropout(params['dropout_rate'] * 0.5))  # Reduced dropout for deeper layers
        
        # Output layer
        model.add(Dense(1, activation='sigmoid'))
        
        # Configure optimizer
        optimizer_name = params.get('optimizer', 'adam')
        if optimizer_name == 'adam':
            optimizer = Adam(learning_rate=params['learning_rate'])
        elif optimizer_name == 'rmsprop':
            optimizer = RMSprop(learning_rate=params['learning_rate'])
        else:  # sgd
            optimizer = SGD(learning_rate=params['learning_rate'], momentum=0.9)
        
        model.compile(
            optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        return model
    
    def optimize_single_symbol(
        self, 
        symbol: str, 
        n_trials: int = 50,
        timeout: int = 1800,
        resume_study: bool = True
    ) -> OptimizationResult:
        """Optimize hyperparameters for a single symbol"""
        
        print(f"\n🎯 Starting optimization for {symbol}")
        print(f"Trials: {n_trials}, Timeout: {timeout}s, Resume: {resume_study}")
        
        # Create or resume study
        study = self.study_manager.create_or_resume_study(
            symbol=symbol,
            resume_if_exists=resume_study
        )
        
        # Define objective function for this symbol
        def objective(trial):
            return self._symbol_objective(trial, symbol)
        
        # Run optimization
        try:
            study.optimize(
                objective,
                n_trials=n_trials,
                timeout=timeout,
                show_progress_bar=True
            )
        except KeyboardInterrupt:
            print(f"\n⚠️ Optimization interrupted for {symbol}")
        
        # Save study
        self.study_manager.save_study(symbol, study)
        
        # Create optimization result
        if study.best_trial:
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            
            result = OptimizationResult(
                symbol=symbol,
                timestamp=timestamp,
                objective_value=study.best_value,
                best_params=study.best_params,
                mean_accuracy=study.best_trial.user_attrs.get('mean_accuracy', 0.0),
                mean_sharpe=study.best_trial.user_attrs.get('mean_sharpe', 0.0),
                std_accuracy=study.best_trial.user_attrs.get('std_accuracy', 0.0),
                std_sharpe=study.best_trial.user_attrs.get('std_sharpe', 0.0),
                num_features=study.best_trial.user_attrs.get('num_features', 0),
                total_trials=len(study.trials),
                completed_trials=len([t for t in study.trials if t.state == TrialState.COMPLETE]),
                study_name=study.study_name
            )
            
            # Save result
            self._save_optimization_result(result)
            self.optimization_results[symbol] = result
            
            # Calculate benchmark metrics
            benchmark = self.opt_manager.calculate_benchmark_metrics(symbol, study.best_value)
            self.opt_manager.benchmark_results[symbol] = benchmark
            
            print(f"\n✅ {symbol} optimization completed:")
            print(f"  Best objective: {study.best_value:.6f}")
            print(f"  Trials completed: {result.completed_trials}/{result.total_trials}")
            print(f"  Improvement vs previous: {benchmark.improvement:.6f}")
            
            return result
        else:
            logger.error(f"No successful trials for {symbol}")
            return None
    
    def _symbol_objective(self, trial: optuna.Trial, symbol: str) -> float:
        """Objective function for a specific symbol"""
        try:
            # Get hyperparameters
            params = self.suggest_advanced_hyperparameters(trial, symbol)
            
            # Note: This is a placeholder for the actual data preparation and training
            # The full implementation would include data loading, feature engineering,
            # model training, and cross-validation
            
            # For now, return a dummy objective value
            return 0.5 + np.random.random() * 0.3  # Placeholder
            
        except Exception as e:
            logger.error(f"Trial failed for {symbol}: {e}")
            return -1.0
        finally:
            # Clean up memory
            tf.keras.backend.clear_session()
    
    def _save_optimization_result(self, result: OptimizationResult):
        """Save optimization result to files"""
        # Save best parameters
        params_file = Path(RESULTS_PATH) / f"best_params_{result.symbol}_{result.timestamp}.json"
        
        params_data = {
            'symbol': result.symbol,
            'timestamp': result.timestamp,
            'objective_value': result.objective_value,
            'mean_accuracy': result.mean_accuracy,
            'mean_sharpe': result.mean_sharpe,
            'num_features': result.num_features,
            'total_trials': result.total_trials,
            'completed_trials': result.completed_trials,
            'best_params': result.best_params
        }
        
        with open(params_file, 'w') as f:
            json.dump(params_data, f, indent=2)
        
        logger.info(f"Saved optimization result: {params_file}")

# Initialize multi-symbol optimizer
multi_optimizer = MultiSymbolOptimizer(opt_manager, study_manager)

print("\n✅ Multi-Symbol Optimizer Ready!")
print("Features:")
print("  - Enhanced hyperparameter space")
print("  - Symbol-specific parameter adjustments")
print("  - Advanced model architectures")
print("  - Multi-objective optimization")
print("  - Comprehensive result tracking")


✅ Multi-Symbol Optimizer Ready!
Features:
  - Enhanced hyperparameter space
  - Symbol-specific parameter adjustments
  - Advanced model architectures
  - Multi-objective optimization
  - Comprehensive result tracking


## 6. Multi-Symbol Optimization Execution

In [10]:
class MultiSymbolExecutor:
    """Execute optimization across multiple symbols with intelligent scheduling"""
    
    def __init__(self, multi_optimizer: MultiSymbolOptimizer):
        self.multi_optimizer = multi_optimizer
        self.execution_results: Dict[str, OptimizationResult] = {}
        self.execution_log: List[Dict[str, Any]] = []
        
    def run_parallel_optimization(
        self,
        symbols: List[str] = None,
        n_trials_per_symbol: int = 30,
        timeout_per_symbol: int = 1200,
        priority_symbols: List[str] = None
    ) -> Dict[str, OptimizationResult]:
        """Run optimization across multiple symbols with intelligent prioritization"""
        
        if symbols is None:
            symbols = SYMBOLS
        
        # Prioritize symbols based on historical performance and data quality
        prioritized_symbols = self._prioritize_symbols(symbols, priority_symbols)
        
        print(f"🚀 Starting multi-symbol optimization")
        print(f"Symbol order: {prioritized_symbols}")
        print(f"Trials per symbol: {n_trials_per_symbol}")
        print(f"Timeout per symbol: {timeout_per_symbol}s")
        
        total_start_time = datetime.now()
        
        for i, symbol in enumerate(prioritized_symbols):
            print(f"\n{'='*60}")
            print(f"📊 Optimizing {symbol} ({i+1}/{len(prioritized_symbols)})")
            print(f"{'='*60}")
            
            symbol_start_time = datetime.now()
            
            try:
                # Run optimization for this symbol
                result = self.multi_optimizer.optimize_single_symbol(
                    symbol=symbol,
                    n_trials=n_trials_per_symbol,
                    timeout=timeout_per_symbol,
                    resume_study=True
                )
                
                if result:
                    self.execution_results[symbol] = result
                    
                    # Log execution details
                    execution_time = (datetime.now() - symbol_start_time).total_seconds()
                    self.execution_log.append({
                        'symbol': symbol,
                        'timestamp': datetime.now().isoformat(),
                        'execution_time': execution_time,
                        'objective_value': result.objective_value,
                        'completed_trials': result.completed_trials,
                        'total_trials': result.total_trials,
                        'status': 'success'
                    })\n                    \n                    print(f\"\\n✅ {symbol} completed successfully in {execution_time:.1f}s\")\n                    print(f\"   Objective: {result.objective_value:.6f}\")\n                    print(f\"   Trials: {result.completed_trials}/{result.total_trials}\")\n                    \n                else:\n                    self.execution_log.append({\n                        'symbol': symbol,\n                        'timestamp': datetime.now().isoformat(),\n                        'execution_time': (datetime.now() - symbol_start_time).total_seconds(),\n                        'status': 'failed',\n                        'error': 'No successful trials'\n                    })\n                    print(f\"\\n❌ {symbol} failed - no successful trials\")\n                    \n            except Exception as e:\n                execution_time = (datetime.now() - symbol_start_time).total_seconds()\n                self.execution_log.append({\n                    'symbol': symbol,\n                    'timestamp': datetime.now().isoformat(),\n                    'execution_time': execution_time,\n                    'status': 'error',\n                    'error': str(e)\n                })\n                print(f\"\\n❌ {symbol} failed with error: {e}\")\n                logger.error(f\"Symbol {symbol} optimization failed: {e}\")\n        \n        total_execution_time = (datetime.now() - total_start_time).total_seconds()\n        \n        print(f\"\\n🎉 Multi-symbol optimization completed!\")\n        print(f\"Total execution time: {total_execution_time:.1f}s\")\n        print(f\"Successful symbols: {len(self.execution_results)}/{len(prioritized_symbols)}\")\n        \n        # Save execution summary\n        self._save_execution_summary(total_execution_time)\n        \n        return self.execution_results\n    \n    def _prioritize_symbols(self, symbols: List[str], priority_symbols: List[str] = None) -> List[str]:\n        \"\"\"Intelligently prioritize symbols for optimization\"\"\"\n        \n        if priority_symbols:\n            # Use provided priority list\n            prioritized = [s for s in priority_symbols if s in symbols]\n            remaining = [s for s in symbols if s not in prioritized]\n            return prioritized + remaining\n        \n        # Automatic prioritization based on:\n        # 1. Historical optimization performance\n        # 2. Data quality\n        # 3. Market importance\n        \n        symbol_scores = {}\n        \n        for symbol in symbols:\n            score = 0\n            \n            # Historical performance weight\n            if symbol in self.multi_optimizer.opt_manager.optimization_history:\n                historical_results = self.multi_optimizer.opt_manager.optimization_history[symbol]\n                if historical_results:\n                    best_historical = max(r.objective_value for r in historical_results)\n                    score += best_historical * 10  # Weight historical performance\n            \n            # Data quality weight\n            if symbol in all_indicators:\n                data_quality = len(all_indicators[symbol]) / 5000  # Normalize by expected data size\n                score += min(data_quality, 1.0) * 5\n            \n            # Market importance weight (major pairs first)\n            major_pairs = ['EURUSD', 'GBPUSD', 'USDJPY']\n            if symbol in major_pairs:\n                score += 3\n            \n            # USD pairs preference\n            if 'USD' in symbol:\n                score += 1\n            \n            symbol_scores[symbol] = score\n        \n        # Sort by score (descending)\n        prioritized = sorted(symbols, key=lambda s: symbol_scores.get(s, 0), reverse=True)\n        \n        print(f\"\\n📊 Symbol Prioritization:\")\n        for i, symbol in enumerate(prioritized):\n            score = symbol_scores.get(symbol, 0)\n            print(f\"  {i+1}. {symbol}: {score:.2f} points\")\n        \n        return prioritized\n    \n    def run_transfer_learning_optimization(\n        self,\n        source_symbol: str = 'EURUSD',\n        target_symbols: List[str] = None,\n        n_trials_transfer: int = 20,\n        n_trials_fine_tune: int = 30\n    ) -> Dict[str, OptimizationResult]:\n        \"\"\"Run optimization using parameter transfer from a source symbol\"\"\"\n        \n        if target_symbols is None:\n            target_symbols = [s for s in SYMBOLS if s != source_symbol]\n        \n        print(f\"🔄 Starting transfer learning optimization\")\n        print(f\"Source symbol: {source_symbol}\")\n        print(f\"Target symbols: {target_symbols}\")\n        \n        # Get best parameters from source symbol\n        source_params = self.multi_optimizer.opt_manager.get_warm_start_params(source_symbol)\n        \n        if source_params is None:\n            print(f\"❌ No parameters available for source symbol {source_symbol}\")\n            print(f\"Running standard optimization for {source_symbol} first...\")\n            \n            # Optimize source symbol first\n            source_result = self.multi_optimizer.optimize_single_symbol(\n                symbol=source_symbol,\n                n_trials=50,\n                timeout=1800\n            )\n            \n            if source_result:\n                source_params = source_result.best_params\n            else:\n                print(f\"❌ Failed to optimize source symbol {source_symbol}\")\n                return {}\n        \n        print(f\"✅ Using parameters from {source_symbol} as baseline\")\n        \n        transfer_results = {}\n        \n        for target_symbol in target_symbols:\n            print(f\"\\n🎯 Transfer learning: {source_symbol} → {target_symbol}\")\n            \n            try:\n                # Create study with warm start from source parameters\n                study = self.multi_optimizer.study_manager.create_or_resume_study(\n                    symbol=target_symbol,\n                    resume_if_exists=False  # Create fresh study for transfer\n                )\n                \n                # Add multiple variations of source parameters\n                self._add_transfer_learning_trials(study, source_params, n_trials_transfer)\n                \n                # Run additional optimization\n                def objective(trial):\n                    return self.multi_optimizer._symbol_objective(trial, target_symbol)\n                \n                study.optimize(\n                    objective,\n                    n_trials=n_trials_fine_tune,\n                    show_progress_bar=True\n                )\n                \n                # Create result\n                if study.best_trial:\n                    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')\n                    \n                    result = OptimizationResult(\n                        symbol=target_symbol,\n                        timestamp=timestamp,\n                        objective_value=study.best_value,\n                        best_params=study.best_params,\n                        mean_accuracy=study.best_trial.user_attrs.get('mean_accuracy', 0.0),\n                        mean_sharpe=study.best_trial.user_attrs.get('mean_sharpe', 0.0),\n                        std_accuracy=study.best_trial.user_attrs.get('std_accuracy', 0.0),\n                        std_sharpe=study.best_trial.user_attrs.get('std_sharpe', 0.0),\n                        num_features=study.best_trial.user_attrs.get('num_features', 0),\n                        total_trials=len(study.trials),\n                        completed_trials=len([t for t in study.trials if t.state == TrialState.COMPLETE]),\n                        study_name=f\"transfer_{source_symbol}_to_{target_symbol}_{timestamp}\"\n                    )\n                    \n                    transfer_results[target_symbol] = result\n                    self.multi_optimizer._save_optimization_result(result)\n                    \n                    print(f\"  ✅ Transfer completed: {study.best_value:.6f}\")\n                    \n                else:\n                    print(f\"  ❌ Transfer failed for {target_symbol}\")\n                    \n            except Exception as e:\n                print(f\"  ❌ Transfer error for {target_symbol}: {e}\")\n                logger.error(f\"Transfer learning failed for {target_symbol}: {e}\")\n        \n        print(f\"\\n🎉 Transfer learning completed!\")\n        print(f\"Successful transfers: {len(transfer_results)}/{len(target_symbols)}\")\n        \n        return transfer_results\n    \n    def _add_transfer_learning_trials(\n        self, \n        study: optuna.Study, \n        source_params: Dict[str, Any], \n        n_variations: int = 10\n    ):\n        \"\"\"Add parameter variations for transfer learning\"\"\"\n        \n        # Add exact source parameters\n        try:\n            study.enqueue_trial(source_params)\n            print(f\"    📌 Enqueued exact source parameters\")\n        except Exception as e:\n            print(f\"    ⚠️ Failed to enqueue exact parameters: {e}\")\n        \n        # Add variations\n        for i in range(n_variations):\n            try:\n                variation_factor = 0.05 + i * 0.03  # Increasing variation\n                varied_params = self.multi_optimizer.study_manager.create_parameter_variation(\n                    source_params, variation_factor\n                )\n                study.enqueue_trial(varied_params)\n                print(f\"    📌 Enqueued variation {i+1} (factor: {variation_factor:.2f})\")\n            except Exception as e:\n                print(f\"    ⚠️ Failed to enqueue variation {i+1}: {e}\")\n    \n    def _save_execution_summary(self, total_time: float):\n        \"\"\"Save execution summary to file\"\"\"\n        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')\n        summary_file = Path(RESULTS_PATH) / f\"multi_symbol_execution_{timestamp}.json\"\n        \n        summary_data = {\n            'timestamp': timestamp,\n            'total_execution_time': total_time,\n            'symbols_attempted': len(self.execution_log),\n            'symbols_successful': len(self.execution_results),\n            'execution_log': self.execution_log,\n            'results_summary': {\n                symbol: {\n                    'objective_value': result.objective_value,\n                    'mean_accuracy': result.mean_accuracy,\n                    'mean_sharpe': result.mean_sharpe,\n                    'completed_trials': result.completed_trials\n                }\n                for symbol, result in self.execution_results.items()\n            }\n        }\n        \n        with open(summary_file, 'w') as f:\n            json.dump(summary_data, f, indent=2)\n        \n        print(f\"\\n💾 Execution summary saved: {summary_file}\")\n\n# Initialize executor\nmax_executor = MultiSymbolExecutor(multi_optimizer)\n\nprint(\"✅ Multi-Symbol Executor Ready!\")\nprint(\"Features:\")\nprint(\"  - Intelligent symbol prioritization\")\nprint(\"  - Transfer learning optimization\")\nprint(\"  - Comprehensive execution tracking\")\nprint(\"  - Automatic error handling and recovery\")"

SyntaxError: unexpected character after line continuation character (3037801512.py, line 60)

## 7. Comprehensive Benchmarking Dashboard

In [ ]:
class BenchmarkingDashboard:
    \"\"\"Comprehensive benchmarking and analysis dashboard\"\"\"\n    \n    def __init__(self, opt_manager: AdvancedOptimizationManager):\n        self.opt_manager = opt_manager\n        self.dashboard_data: Dict[str, Any] = {}\n        \n    def generate_comprehensive_report(self, output_dir: str = \"reports\") -> str:\n        \"\"\"Generate a comprehensive benchmarking report\"\"\"\n        \n        print(\"📊 Generating comprehensive benchmarking report...\")\n        \n        # Create output directory\n        report_dir = Path(output_dir)\n        report_dir.mkdir(exist_ok=True)\n        \n        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')\n        report_file = report_dir / f\"optimization_benchmark_report_{timestamp}.html\"\n        \n        # Collect all data\n        self._collect_dashboard_data()\n        \n        # Generate HTML report\n        html_content = self._generate_html_report()\n        \n        with open(report_file, 'w') as f:\n            f.write(html_content)\n        \n        print(f\"✅ Comprehensive report generated: {report_file}\")\n        return str(report_file)\n    \n    def _collect_dashboard_data(self):\n        \"\"\"Collect all data for the dashboard\"\"\"\n        \n        print(\"📈 Collecting optimization data...\")\n        \n        # Historical performance summary\n        self.dashboard_data['performance_summary'] = self._get_performance_summary()\n        \n        # Parameter analysis\n        self.dashboard_data['parameter_analysis'] = self._analyze_parameters()\n        \n        # Benchmark comparisons\n        self.dashboard_data['benchmark_comparisons'] = self._get_benchmark_comparisons()\n        \n        # Trend analysis\n        self.dashboard_data['trend_analysis'] = self._analyze_trends()\n        \n        # Success rate analysis\n        self.dashboard_data['success_analysis'] = self._analyze_success_rates()\n        \n    def _get_performance_summary(self) -> Dict[str, Any]:\n        \"\"\"Get performance summary across all symbols\"\"\"\n        \n        summary = {\n            'total_symbols': len(SYMBOLS),\n            'optimized_symbols': len(self.opt_manager.optimization_history),\n            'symbol_details': {}\n        }\n        \n        for symbol in SYMBOLS:\n            if symbol in self.opt_manager.optimization_history:\n                results = self.opt_manager.optimization_history[symbol]\n                \n                if results:\n                    best_result = max(results, key=lambda r: r.objective_value)\n                    latest_result = max(results, key=lambda r: r.timestamp)\n                    \n                    summary['symbol_details'][symbol] = {\n                        'total_runs': len(results),\n                        'best_objective': best_result.objective_value,\n                        'best_accuracy': best_result.mean_accuracy,\n                        'best_sharpe': best_result.mean_sharpe,\n                        'latest_objective': latest_result.objective_value,\n                        'latest_timestamp': latest_result.timestamp,\n                        'improvement_trend': self._calculate_improvement_trend(results)\n                    }\n                else:\n                    summary['symbol_details'][symbol] = {\n                        'total_runs': 0,\n                        'status': 'no_data'\n                    }\n            else:\n                summary['symbol_details'][symbol] = {\n                    'total_runs': 0,\n                    'status': 'not_optimized'\n                }\n        \n        return summary\n    \n    def _analyze_parameters(self) -> Dict[str, Any]:\n        \"\"\"Analyze parameter patterns across successful optimizations\"\"\"\n        \n        parameter_analysis = {\n            'successful_patterns': {},\n            'parameter_distributions': {},\n            'correlation_analysis': {}\n        }\n        \n        # Collect all successful parameters\n        all_params = []\n        all_objectives = []\n        \n        for symbol, results in self.opt_manager.optimization_history.items():\n            for result in results:\n                if result.objective_value > 0:  # Successful optimization\n                    all_params.append(result.best_params)\n                    all_objectives.append(result.objective_value)\n        \n        if all_params:\n            # Find common parameter patterns\n            param_keys = set()\n            for params in all_params:\n                param_keys.update(params.keys())\n            \n            for param_key in param_keys:\n                values = [params.get(param_key) for params in all_params if param_key in params]\n                if values and all(isinstance(v, (int, float)) for v in values):\n                    parameter_analysis['parameter_distributions'][param_key] = {\n                        'mean': np.mean(values),\n                        'std': np.std(values),\n                        'min': np.min(values),\n                        'max': np.max(values),\n                        'median': np.median(values)\n                    }\n            \n            # Correlation with objective values\n            for param_key in param_keys:\n                values = [params.get(param_key) for params in all_params if param_key in params]\n                if len(values) == len(all_objectives) and all(isinstance(v, (int, float)) for v in values):\n                    correlation = np.corrcoef(values, all_objectives[:len(values)])[0, 1]\n                    if not np.isnan(correlation):\n                        parameter_analysis['correlation_analysis'][param_key] = correlation\n        \n        return parameter_analysis\n    \n    def _get_benchmark_comparisons(self) -> Dict[str, Any]:\n        \"\"\"Compare current results with benchmarks\"\"\"\n        \n        comparisons = {\n            'symbol_rankings': [],\n            'improvement_metrics': {},\n            'competitive_analysis': {}\n        }\n        \n        # Rank symbols by best performance\n        symbol_scores = []\n        for symbol in SYMBOLS:\n            if symbol in self.opt_manager.optimization_history:\n                results = self.opt_manager.optimization_history[symbol]\n                if results:\n                    best_score = max(r.objective_value for r in results)\n                    symbol_scores.append((symbol, best_score))\n        \n        # Sort by score (descending)\n        symbol_scores.sort(key=lambda x: x[1], reverse=True)\n        comparisons['symbol_rankings'] = symbol_scores\n        \n        # Calculate improvement metrics\n        for symbol, results in self.opt_manager.optimization_history.items():\n            if len(results) >= 2:\n                sorted_results = sorted(results, key=lambda r: r.timestamp)\n                first_score = sorted_results[0].objective_value\n                latest_score = sorted_results[-1].objective_value\n                improvement = latest_score - first_score\n                improvement_pct = (improvement / abs(first_score)) * 100 if first_score != 0 else 0\n                \n                comparisons['improvement_metrics'][symbol] = {\n                    'absolute_improvement': improvement,\n                    'percentage_improvement': improvement_pct,\n                    'total_runs': len(results)\n                }\n        \n        return comparisons\n    \n    def _analyze_trends(self) -> Dict[str, Any]:\n        \"\"\"Analyze optimization trends over time\"\"\"\n        \n        trends = {\n            'temporal_patterns': {},\n            'convergence_analysis': {},\n            'seasonal_effects': {}\n        }\n        \n        for symbol, results in self.opt_manager.optimization_history.items():\n            if len(results) >= 3:\n                # Sort by timestamp\n                sorted_results = sorted(results, key=lambda r: r.timestamp)\n                \n                # Calculate moving averages\n                scores = [r.objective_value for r in sorted_results]\n                timestamps = [r.timestamp for r in sorted_results]\n                \n                trends['temporal_patterns'][symbol] = {\n                    'scores': scores,\n                    'timestamps': timestamps,\n                    'trend_direction': 'improving' if scores[-1] > scores[0] else 'declining',\n                    'volatility': np.std(scores) if len(scores) > 1 else 0\n                }\n        \n        return trends\n    \n    def _analyze_success_rates(self) -> Dict[str, Any]:\n        \"\"\"Analyze success rates and failure patterns\"\"\"\n        \n        success_analysis = {\n            'overall_stats': {},\n            'failure_patterns': {},\n            'success_factors': {}\n        }\n        \n        total_optimizations = sum(len(results) for results in self.opt_manager.optimization_history.values())\n        successful_optimizations = sum(\n            len([r for r in results if r.objective_value > 0]) \n            for results in self.opt_manager.optimization_history.values()\n        )\n        \n        success_analysis['overall_stats'] = {\n            'total_optimizations': total_optimizations,\n            'successful_optimizations': successful_optimizations,\n            'success_rate': successful_optimizations / total_optimizations if total_optimizations > 0 else 0,\n            'symbols_with_success': len([\n                symbol for symbol, results in self.opt_manager.optimization_history.items()\n                if any(r.objective_value > 0 for r in results)\n            ])\n        }\n        \n        return success_analysis\n    \n    def _calculate_improvement_trend(self, results: List[OptimizationResult]) -> str:\n        \"\"\"Calculate improvement trend for a symbol\"\"\"\n        \n        if len(results) < 2:\n            return 'insufficient_data'\n        \n        sorted_results = sorted(results, key=lambda r: r.timestamp)\n        recent_scores = [r.objective_value for r in sorted_results[-3:]]  # Last 3 results\n        \n        if len(recent_scores) >= 2:\n            if recent_scores[-1] > recent_scores[0]:\n                return 'improving'\n            elif recent_scores[-1] < recent_scores[0]:\n                return 'declining'\n            else:\n                return 'stable'\n        \n        return 'unknown'\n    \n    def _generate_html_report(self) -> str:\n        \"\"\"Generate HTML report content\"\"\"\n        \n        html_template = \"\"\"\n<!DOCTYPE html>\n<html>\n<head>\n    <title>Advanced Hyperparameter Optimization Benchmark Report</title>\n    <style>\n        body {{ font-family: Arial, sans-serif; margin: 20px; background-color: #f5f5f5; }}\n        .container {{ max-width: 1200px; margin: 0 auto; background-color: white; padding: 20px; border-radius: 10px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); }}\n        .header {{ text-align: center; color: #2c3e50; margin-bottom: 30px; }}\n        .section {{ margin-bottom: 30px; padding: 20px; border-left: 4px solid #3498db; background-color: #f8f9fa; }}\n        .metric {{ display: inline-block; margin: 10px 20px; padding: 15px; background-color: white; border-radius: 5px; border: 1px solid #ddd; }}\n        .metric-label {{ font-weight: bold; color: #2c3e50; }}\n        .metric-value {{ font-size: 1.2em; color: #27ae60; }}\n        .symbol-grid {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); gap: 20px; }}\n        .symbol-card {{ padding: 15px; border: 1px solid #ddd; border-radius: 5px; background-color: white; }}\n        .symbol-name {{ font-weight: bold; font-size: 1.1em; color: #2c3e50; margin-bottom: 10px; }}\n        .status-success {{ color: #27ae60; }}\n        .status-warning {{ color: #f39c12; }}\n        .status-error {{ color: #e74c3c; }}\n        .trend-improving {{ color: #27ae60; }}\n        .trend-declining {{ color: #e74c3c; }}\n        .trend-stable {{ color: #f39c12; }}\n        table {{ width: 100%; border-collapse: collapse; margin: 10px 0; }}\n        th, td {{ padding: 10px; text-align: left; border-bottom: 1px solid #ddd; }}\n        th {{ background-color: #3498db; color: white; }}\n        .parameter-analysis {{ background-color: #ecf0f1; padding: 15px; border-radius: 5px; }}\n        .footer {{ text-align: center; margin-top: 30px; color: #7f8c8d; font-size: 0.9em; }}\n    </style>\n</head>\n<body>\n    <div class=\"container\">\n        <div class=\"header\">\n            <h1>🚀 Advanced Hyperparameter Optimization Report</h1>\n            <p>Generated on {timestamp}</p>\n        </div>\n        \n        <div class=\"section\">\n            <h2>📊 Overall Performance Summary</h2>\n            <div class=\"metric\">\n                <div class=\"metric-label\">Total Symbols</div>\n                <div class=\"metric-value\">{total_symbols}</div>\n            </div>\n            <div class=\"metric\">\n                <div class=\"metric-label\">Optimized Symbols</div>\n                <div class=\"metric-value\">{optimized_symbols}</div>\n            </div>\n            <div class=\"metric\">\n                <div class=\"metric-label\">Success Rate</div>\n                <div class=\"metric-value\">{success_rate:.1f}%</div>\n            </div>\n            <div class=\"metric\">\n                <div class=\"metric-label\">Total Optimizations</div>\n                <div class=\"metric-value\">{total_optimizations}</div>\n            </div>\n        </div>\n        \n        <div class=\"section\">\n            <h2>🎯 Symbol Performance Details</h2>\n            <div class=\"symbol-grid\">\n                {symbol_cards}\n            </div>\n        </div>\n        \n        <div class=\"section\">\n            <h2>📈 Top Performing Symbols</h2>\n            <table>\n                <tr>\n                    <th>Rank</th>\n                    <th>Symbol</th>\n                    <th>Best Objective</th>\n                    <th>Best Accuracy</th>\n                    <th>Best Sharpe</th>\n                    <th>Status</th>\n                </tr>\n                {ranking_rows}\n            </table>\n        </div>\n        \n        <div class=\"section\">\n            <h2>🔧 Parameter Analysis</h2>\n            <div class=\"parameter-analysis\">\n                {parameter_insights}\n            </div>\n        </div>\n        \n        <div class=\"section\">\n            <h2>📊 Improvement Metrics</h2>\n            <table>\n                <tr>\n                    <th>Symbol</th>\n                    <th>Absolute Improvement</th>\n                    <th>Percentage Improvement</th>\n                    <th>Total Runs</th>\n                </tr>\n                {improvement_rows}\n            </table>\n        </div>\n        \n        <div class=\"footer\">\n            <p>Generated by Advanced Hyperparameter Optimization System</p>\n            <p>🤖 Powered by Optuna and TensorFlow</p>\n        </div>\n    </div>\n</body>\n</html>\n        \"\"\"\n        \n        # Prepare data for template\n        perf_summary = self.dashboard_data['performance_summary']\n        success_analysis = self.dashboard_data['success_analysis']\n        benchmark_comparisons = self.dashboard_data['benchmark_comparisons']\n        parameter_analysis = self.dashboard_data['parameter_analysis']\n        \n        # Generate symbol cards\n        symbol_cards = \"\"\n        for symbol, details in perf_summary['symbol_details'].items():\n            if 'best_objective' in details:\n                trend_class = f\"trend-{details.get('improvement_trend', 'unknown')}\"\n                symbol_cards += f\"\"\"\n                <div class=\"symbol-card\">\n                    <div class=\"symbol-name\">{symbol}</div>\n                    <div>Best Objective: <span class=\"status-success\">{details['best_objective']:.6f}</span></div>\n                    <div>Accuracy: {details['best_accuracy']:.4f}</div>\n                    <div>Sharpe: {details['best_sharpe']:.4f}</div>\n                    <div>Runs: {details['total_runs']}</div>\n                    <div>Trend: <span class=\"{trend_class}\">{details.get('improvement_trend', 'unknown')}</span></div>\n                </div>\n                \"\"\"\n            else:\n                status_class = \"status-error\" if details.get('status') == 'not_optimized' else \"status-warning\"\n                symbol_cards += f\"\"\"\n                <div class=\"symbol-card\">\n                    <div class=\"symbol-name\">{symbol}</div>\n                    <div class=\"{status_class}\">Status: {details.get('status', 'unknown')}</div>\n                </div>\n                \"\"\"\n        \n        # Generate ranking rows\n        ranking_rows = \"\"\n        for i, (symbol, score) in enumerate(benchmark_comparisons['symbol_rankings'][:10]):\n            details = perf_summary['symbol_details'][symbol]\n            ranking_rows += f\"\"\"\n            <tr>\n                <td>{i+1}</td>\n                <td>{symbol}</td>\n                <td>{score:.6f}</td>\n                <td>{details.get('best_accuracy', 0):.4f}</td>\n                <td>{details.get('best_sharpe', 0):.4f}</td>\n                <td class=\"status-success\">Optimized</td>\n            </tr>\n            \"\"\"\n        \n        # Generate parameter insights\n        param_insights = \"<h3>Parameter Distribution Analysis</h3>\"\n        for param, stats in parameter_analysis.get('parameter_distributions', {}).items():\n            param_insights += f\"\"\"\n            <p><strong>{param}:</strong> Mean: {stats['mean']:.4f}, Std: {stats['std']:.4f}, Range: [{stats['min']:.4f}, {stats['max']:.4f}]</p>\n            \"\"\"\n        \n        param_insights += \"<h3>Parameter-Objective Correlations</h3>\"\n        for param, correlation in parameter_analysis.get('correlation_analysis', {}).items():\n            correlation_strength = \"Strong\" if abs(correlation) > 0.5 else \"Moderate\" if abs(correlation) > 0.3 else \"Weak\"\n            param_insights += f\"\"\"\n            <p><strong>{param}:</strong> {correlation:.4f} ({correlation_strength})</p>\n            \"\"\"\n        \n        # Generate improvement rows\n        improvement_rows = \"\"\n        for symbol, metrics in benchmark_comparisons.get('improvement_metrics', {}).items():\n            improvement_rows += f\"\"\"\n            <tr>\n                <td>{symbol}</td>\n                <td>{metrics['absolute_improvement']:.6f}</td>\n                <td>{metrics['percentage_improvement']:.2f}%</td>\n                <td>{metrics['total_runs']}</td>\n            </tr>\n            \"\"\"\n        \n        # Fill template\n        return html_template.format(\n            timestamp=datetime.now().strftime('%Y-%m-%d %H:%M:%S'),\n            total_symbols=perf_summary['total_symbols'],\n            optimized_symbols=perf_summary['optimized_symbols'],\n            success_rate=success_analysis['overall_stats'].get('success_rate', 0) * 100,\n            total_optimizations=success_analysis['overall_stats'].get('total_optimizations', 0),\n            symbol_cards=symbol_cards,\n            ranking_rows=ranking_rows,\n            parameter_insights=param_insights,\n            improvement_rows=improvement_rows\n        )\n    \n    def create_interactive_plots(self, output_dir: str = \"plots\"):\n        \"\"\"Create interactive plots for optimization analysis\"\"\"\n        \n        print(\"📈 Creating interactive optimization plots...\")\n        \n        plot_dir = Path(output_dir)\n        plot_dir.mkdir(exist_ok=True)\n        \n        # Performance comparison plot\n        self._create_performance_comparison_plot(plot_dir)\n        \n        # Parameter correlation heatmap\n        self._create_parameter_correlation_plot(plot_dir)\n        \n        # Optimization timeline\n        self._create_optimization_timeline_plot(plot_dir)\n        \n        print(f\"✅ Interactive plots saved to: {plot_dir}\")\n    \n    def _create_performance_comparison_plot(self, output_dir: Path):\n        \"\"\"Create performance comparison plot\"\"\"\n        \n        plt.figure(figsize=(12, 8))\n        \n        symbols = []\n        best_scores = []\n        colors = []\n        \n        for symbol in SYMBOLS:\n            if symbol in self.opt_manager.optimization_history:\n                results = self.opt_manager.optimization_history[symbol]\n                if results:\n                    best_score = max(r.objective_value for r in results)\n                    symbols.append(symbol)\n                    best_scores.append(best_score)\n                    colors.append('#27ae60' if best_score > 0.5 else '#f39c12' if best_score > 0 else '#e74c3c')\n        \n        plt.bar(symbols, best_scores, color=colors)\n        plt.title('Best Optimization Scores by Symbol', fontsize=16, fontweight='bold')\n        plt.xlabel('Currency Pairs', fontsize=12)\n        plt.ylabel('Best Objective Value', fontsize=12)\n        plt.xticks(rotation=45)\n        plt.grid(True, alpha=0.3)\n        \n        # Add value labels on bars\n        for i, v in enumerate(best_scores):\n            plt.text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom', fontweight='bold')\n        \n        plt.tight_layout()\n        plt.savefig(output_dir / 'performance_comparison.png', dpi=300, bbox_inches='tight')\n        plt.close()\n    \n    def _create_parameter_correlation_plot(self, output_dir: Path):\n        \"\"\"Create parameter correlation heatmap\"\"\"\n        \n        # Collect parameter data\n        param_data = []\n        for results in self.opt_manager.optimization_history.values():\n            for result in results:\n                if result.objective_value > 0:\n                    param_data.append(result.best_params)\n        \n        if param_data:\n            # Convert to DataFrame\n            df = pd.DataFrame(param_data)\n            \n            # Select numerical columns\n            numerical_cols = df.select_dtypes(include=[np.number]).columns\n            if len(numerical_cols) > 1:\n                correlation_matrix = df[numerical_cols].corr()\n                \n                plt.figure(figsize=(12, 10))\n                sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,\n                           square=True, linewidths=0.5, cbar_kws={\"shrink\": .5})\n                plt.title('Parameter Correlation Matrix', fontsize=16, fontweight='bold')\n                plt.tight_layout()\n                plt.savefig(output_dir / 'parameter_correlation.png', dpi=300, bbox_inches='tight')\n                plt.close()\n    \n    def _create_optimization_timeline_plot(self, output_dir: Path):\n        \"\"\"Create optimization timeline plot\"\"\"\n        \n        plt.figure(figsize=(14, 8))\n        \n        for symbol, results in self.opt_manager.optimization_history.items():\n            if results:\n                timestamps = [datetime.strptime(r.timestamp, '%Y%m%d_%H%M%S') for r in results]\n                objectives = [r.objective_value for r in results]\n                \n                plt.plot(timestamps, objectives, marker='o', label=symbol, linewidth=2, markersize=6)\n        \n        plt.title('Optimization Progress Timeline', fontsize=16, fontweight='bold')\n        plt.xlabel('Time', fontsize=12)\n        plt.ylabel('Objective Value', fontsize=12)\n        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')\n        plt.grid(True, alpha=0.3)\n        plt.xticks(rotation=45)\n        \n        plt.tight_layout()\n        plt.savefig(output_dir / 'optimization_timeline.png', dpi=300, bbox_inches='tight')\n        plt.close()\n\n# Initialize benchmarking dashboard\nbenchmark_dashboard = BenchmarkingDashboard(opt_manager)\n\nprint(\"✅ Benchmarking Dashboard Ready!\")\nprint(\"Features:\")\nprint(\"  - Comprehensive HTML reports\")\nprint(\"  - Interactive performance plots\")\nprint(\"  - Parameter correlation analysis\")\nprint(\"  - Trend and improvement tracking\")\nprint(\"  - Success rate analysis\")"

## 8. Execution Examples and Usage Guide

In [ ]:
# ========================================\n# USAGE EXAMPLES - Choose your optimization approach\n# ========================================\n\nprint(\"🚀 Advanced Hyperparameter Optimization System Ready!\")\nprint(\"\\nChoose your optimization approach:\")\nprint(\"\\n1️⃣  QUICK TEST (Single Symbol - 10 trials)\")\nprint(\"2️⃣  STANDARD OPTIMIZATION (All symbols - 30 trials each)\")\nprint(\"3️⃣  INTENSIVE OPTIMIZATION (All symbols - 50+ trials each)\")\nprint(\"4️⃣  TRANSFER LEARNING (EURUSD → other symbols)\")\nprint(\"5️⃣  BENCHMARK REPORT ONLY (Generate analysis of existing results)\")\n\n# Uncomment the section you want to run:\n\n# ========================================\n# 1️⃣ QUICK TEST - Single Symbol\n# ========================================\n\"\"\"\nprint(\"\\n🎯 Running QUICK TEST on EURUSD...\")\n\n# Single symbol optimization\nresult = multi_optimizer.optimize_single_symbol(\n    symbol='EURUSD',\n    n_trials=10,\n    timeout=600,  # 10 minutes\n    resume_study=True\n)\n\nif result:\n    print(f\"\\n✅ Quick test completed!\")\n    print(f\"Best objective: {result.objective_value:.6f}\")\n    print(f\"Best accuracy: {result.mean_accuracy:.4f}\")\n    print(f\"Best Sharpe: {result.mean_sharpe:.4f}\")\nelse:\n    print(\"❌ Quick test failed\")\n\"\"\"\n\n# ========================================\n# 2️⃣ STANDARD OPTIMIZATION - All Symbols\n# ========================================\n\"\"\"\nprint(\"\\n🎯 Running STANDARD OPTIMIZATION on all symbols...\")\n\n# Multi-symbol optimization with moderate settings\nresults = max_executor.run_parallel_optimization(\n    symbols=SYMBOLS,\n    n_trials_per_symbol=30,\n    timeout_per_symbol=1200,  # 20 minutes per symbol\n    priority_symbols=['EURUSD', 'GBPUSD', 'USDJPY']  # Major pairs first\n)\n\nprint(f\"\\n✅ Standard optimization completed!\")\nprint(f\"Successful symbols: {len(results)}/{len(SYMBOLS)}\")\n\nfor symbol, result in results.items():\n    print(f\"  {symbol}: {result.objective_value:.6f} (accuracy: {result.mean_accuracy:.4f})\")\n\"\"\"\n\n# ========================================\n# 3️⃣ INTENSIVE OPTIMIZATION - Maximum Performance\n# ========================================\n\"\"\"\nprint(\"\\n🎯 Running INTENSIVE OPTIMIZATION...\")\nprint(\"⚠️  This will take several hours but provides the best results\")\n\n# Intensive optimization with maximum trials\nresults = max_executor.run_parallel_optimization(\n    symbols=SYMBOLS,\n    n_trials_per_symbol=75,  # More trials for better optimization\n    timeout_per_symbol=2400,  # 40 minutes per symbol\n    priority_symbols=['EURUSD', 'GBPUSD', 'USDJPY', 'AUDUSD']  # Focus on major pairs\n)\n\nprint(f\"\\n✅ Intensive optimization completed!\")\nprint(f\"Total execution time: {sum(log['execution_time'] for log in max_executor.execution_log):.1f}s\")\n\"\"\"\n\n# ========================================\n# 4️⃣ TRANSFER LEARNING - Parameter Transfer\n# ========================================\n\"\"\"\nprint(\"\\n🎯 Running TRANSFER LEARNING optimization...\")\n\n# First ensure EURUSD is optimized\nif 'EURUSD' not in opt_manager.best_parameters:\n    print(\"📊 Optimizing EURUSD first as source for transfer learning...\")\n    eurusd_result = multi_optimizer.optimize_single_symbol(\n        symbol='EURUSD',\n        n_trials=50,\n        timeout=1800\n    )\n    print(f\"EURUSD baseline: {eurusd_result.objective_value:.6f}\")\n\n# Transfer learning to other symbols\ntransfer_results = max_executor.run_transfer_learning_optimization(\n    source_symbol='EURUSD',\n    target_symbols=['GBPUSD', 'USDJPY', 'AUDUSD', 'USDCAD'],\n    n_trials_transfer=15,  # Initial transfer trials\n    n_trials_fine_tune=35  # Fine-tuning trials\n)\n\nprint(f\"\\n✅ Transfer learning completed!\")\nfor symbol, result in transfer_results.items():\n    print(f\"  {symbol}: {result.objective_value:.6f} (transferred from EURUSD)\")\n\"\"\"\n\n# ========================================\n# 5️⃣ BENCHMARK REPORT - Analysis Only\n# ========================================\n\nprint(\"\\n📊 Generating comprehensive benchmark report...\")\n\n# Generate HTML report\nreport_file = benchmark_dashboard.generate_comprehensive_report(output_dir=\"reports\")\nprint(f\"✅ Report generated: {report_file}\")\n\n# Create interactive plots\nbenchmark_dashboard.create_interactive_plots(output_dir=\"plots\")\nprint(f\"✅ Interactive plots created in: plots/\")\n\n# Display summary statistics\nprint(\"\\n📈 Quick Summary:\")\nif opt_manager.optimization_history:\n    total_runs = sum(len(results) for results in opt_manager.optimization_history.values())\n    successful_runs = sum(\n        len([r for r in results if r.objective_value > 0]) \n        for results in opt_manager.optimization_history.values()\n    )\n    \n    print(f\"  Total optimization runs: {total_runs}\")\n    print(f\"  Successful runs: {successful_runs}\")\n    print(f\"  Success rate: {successful_runs/total_runs*100:.1f}%\" if total_runs > 0 else \"  Success rate: N/A\")\n    \n    # Best performing symbols\n    best_performers = []\n    for symbol, results in opt_manager.optimization_history.items():\n        if results:\n            best_score = max(r.objective_value for r in results)\n            best_performers.append((symbol, best_score))\n    \n    best_performers.sort(key=lambda x: x[1], reverse=True)\n    \n    print(f\"\\n🏆 Top 3 Performing Symbols:\")\n    for i, (symbol, score) in enumerate(best_performers[:3]):\n        print(f\"  {i+1}. {symbol}: {score:.6f}\")\nelse:\n    print(\"  No optimization history found.\")\n    print(\"  Run one of the optimization examples above first.\")\n\nprint(\"\\n\" + \"=\"*60)\nprint(\"🎉 ADVANCED HYPERPARAMETER OPTIMIZATION SYSTEM\")\nprint(\"=\"*60)\nprint(\"\\n✨ Key Features Implemented:\")\nprint(\"  ✅ Study resumption and warm start capabilities\")\nprint(\"  ✅ Multi-symbol optimization with intelligent prioritization\")\nprint(\"  ✅ Parameter transfer learning across currency pairs\")\nprint(\"  ✅ Comprehensive benchmarking and analysis dashboard\")\nprint(\"  ✅ Enhanced model architectures and hyperparameter spaces\")\nprint(\"  ✅ Advanced feature selection and data preparation\")\nprint(\"  ✅ Real-time progress tracking and error handling\")\nprint(\"  ✅ Interactive visualization and reporting\")\n\nprint(\"\\n🚀 Next Steps:\")\nprint(\"  1. Uncomment and run your preferred optimization approach above\")\nprint(\"  2. Monitor results in the optimization_results/ directory\")\nprint(\"  3. Review generated reports and plots for insights\")\nprint(\"  4. Use best parameters for production trading models\")\nprint(\"  5. Set up automated reoptimization schedules\")\n\nprint(\"\\n💡 Pro Tips:\")\nprint(\"  - Start with QUICK TEST to verify system works\")\nprint(\"  - Use TRANSFER LEARNING for faster optimization of similar pairs\")\nprint(\"  - Run INTENSIVE OPTIMIZATION overnight for best results\")\nprint(\"  - Generate BENCHMARK REPORTS regularly to track progress\")\nprint(\"  - Check the 'reports/' and 'plots/' directories for detailed analysis\")\n\nprint(f\"\\n📁 Output Directories:\")\nprint(f\"  - Optimization results: {RESULTS_PATH}/\")\nprint(f\"  - Model exports: {MODELS_PATH}/\")\nprint(f\"  - Reports: reports/\")\nprint(f\"  - Plots: plots/\")\nprint(f\"  - Logs: advanced_optimization.log\")"

## 3. Enhanced Data Loading and Preparation

In [ ]:
class AdvancedDataLoader:
    """Enhanced data loader with multi-symbol support and caching"""
    
    def __init__(self, data_path: str):
        self.data_path = Path(data_path)
        self.cached_data: Dict[str, pd.DataFrame] = {}
        self.cached_indicators: Dict[str, pd.DataFrame] = {}
        self.cached_rcs: Optional[pd.DataFrame] = None
        
    def load_all_data(self, symbols: List[str] = None) -> Dict[str, pd.DataFrame]:
        """Load all available forex data"""
        if symbols is None:
            symbols = SYMBOLS
        
        print(f"📊 Loading data for {len(symbols)} symbols...")
        
        data = {}
        for symbol in symbols:
            symbol_data = self.load_symbol_data(symbol)
            if symbol_data is not None:
                data[symbol] = symbol_data
                print(f"  ✅ {symbol}: {len(symbol_data)} rows")
            else:
                print(f"  ❌ {symbol}: Failed to load")
        
        return data
    
    def load_symbol_data(self, symbol: str) -> Optional[pd.DataFrame]:
        """Load data for a specific symbol with caching"""
        if symbol in self.cached_data:
            return self.cached_data[symbol]
        
        # Try different file formats
        file_patterns = [
            f"metatrader_{symbol}.parquet",
            f"metatrader_{symbol}.h5",
            f"{symbol}.parquet",
            f"{symbol}.h5",
            f"{symbol}.csv"
        ]
        
        for pattern in file_patterns:
            file_path = self.data_path / pattern
            if file_path.exists():
                try:
                    if pattern.endswith('.parquet'):
                        df = pd.read_parquet(file_path)
                    elif pattern.endswith('.h5'):
                        df = pd.read_hdf(file_path, key='data')
                    elif pattern.endswith('.csv'):
                        df = pd.read_csv(file_path, index_col=0, parse_dates=True)
                    else:
                        continue
                    
                    # Standardize index
                    if 'time' in df.columns:
                        df = df.set_index('time')
                    elif 'timestamp' in df.columns:
                        df = df.set_index('timestamp')
                    
                    # Ensure datetime index
                    if not isinstance(df.index, pd.DatetimeIndex):
                        df.index = pd.to_datetime(df.index)
                    
                    # Cache and return
                    self.cached_data[symbol] = df
                    return df
                    
                except Exception as e:
                    logger.warning(f"Failed to load {file_path}: {e}")
                    continue
        
        logger.error(f"No data file found for {symbol}")
        return None
    
    def create_multi_index_prices(self, data: Dict[str, pd.DataFrame]) -> pd.DataFrame:
        """Create MultiIndex DataFrame for price data"""
        print("🔄 Creating MultiIndex price structure...")
        
        multi_data = {}
        
        for symbol, df in data.items():
            # Ensure we have OHLC columns
            required_cols = ['open', 'high', 'low', 'close']
            
            for col in required_cols:
                if col in df.columns:
                    multi_data[(symbol, col)] = df[col]
                elif col == 'open' and 'close' in df.columns:
                    # Use close as fallback for open
                    multi_data[(symbol, col)] = df['close']
                elif col in ['high', 'low'] and 'close' in df.columns:
                    # Use close as fallback
                    multi_data[(symbol, col)] = df['close']
                else:
                    logger.warning(f"Missing {col} for {symbol}")
            
            # Add volume if available
            volume_cols = ['tick_volume', 'volume', 'real_volume']
            for vol_col in volume_cols:
                if vol_col in df.columns:
                    multi_data[(symbol, 'tick_volume')] = df[vol_col]
                    break
            else:
                # Create synthetic volume
                multi_data[(symbol, 'tick_volume')] = pd.Series(
                    np.random.randint(100, 2000, len(df)),
                    index=df.index
                )
        
        prices_df = pd.DataFrame(multi_data)
        print(f"✅ MultiIndex prices created: {prices_df.shape}")
        
        return prices_df
    
    def calculate_rcs(self, prices_df: pd.DataFrame) -> pd.DataFrame:
        """Calculate Relative Currency Strength"""
        if self.cached_rcs is not None:
            return self.cached_rcs
        
        print("🧮 Calculating Relative Currency Strength...")
        
        # Extract close prices
        close_prices = {}
        for symbol in SYMBOLS:
            if (symbol, 'close') in prices_df.columns:
                close_prices[symbol] = prices_df[(symbol, 'close')]
        
        if not close_prices:
            logger.warning("No close prices found for RCS calculation")
            return pd.DataFrame()
        
        close_df = pd.DataFrame(close_prices)
        log_returns = np.log(close_df / close_df.shift(1)).dropna()
        
        # Extract unique currencies
        currencies = list(set([s[:3] for s in log_returns.columns] + [s[3:6] for s in log_returns.columns]))
        
        # Calculate RCS
        rcs_data = {c: [] for c in currencies}
        
        for i in range(len(log_returns)):
            row = log_returns.iloc[i]
            daily_strength = {c: 0 for c in currencies}
            counts = {c: 0 for c in currencies}
            
            for pair, ret in row.items():
                if pd.notna(ret):
                    base, quote = pair[:3], pair[3:]
                    daily_strength[base] += ret
                    daily_strength[quote] -= ret
                    counts[base] += 1
                    counts[quote] += 1
            
            for c in currencies:
                avg_strength = daily_strength[c] / counts[c] if counts[c] > 0 else 0
                rcs_data[c].append(avg_strength)
        
        rcs_df = pd.DataFrame(rcs_data, index=log_returns.index)
        self.cached_rcs = rcs_df
        
        print(f"✅ RCS calculated for {len(currencies)} currencies")
        return rcs_df

# Initialize data loader
data_loader = AdvancedDataLoader(DATA_PATH)

# Load all available data
all_data = data_loader.load_all_data(SYMBOLS)
multi_prices = data_loader.create_multi_index_prices(all_data)
rcs_data = data_loader.calculate_rcs(multi_prices)

print(f"\n✅ Data loading completed:")
print(f"  - Loaded data for {len(all_data)} symbols")
print(f"  - MultiIndex prices: {multi_prices.shape}")
print(f"  - RCS data: {rcs_data.shape}")
if not multi_prices.empty:
    print(f"  - Date range: {multi_prices.index.min()} to {multi_prices.index.max()}")
else:
    print("  - No price data loaded - check data directory")

## 4. Enhanced Technical Indicators with Caching

In [ ]:
# Import the existing indicator calculation function from the original notebook
# We'll enhance it with caching and parallel processing

def calculate_enhanced_indicators(prices_df: pd.DataFrame, symbols: List[str], use_cache: bool = True) -> Dict[str, pd.DataFrame]:
    """Calculate enhanced technical indicators with caching"""
    
    # Check cache first
    if use_cache and hasattr(data_loader, 'cached_indicators') and data_loader.cached_indicators:
        print("📊 Using cached indicators...")
        return data_loader.cached_indicators
    
    print("⚙️ Calculating enhanced technical indicators...")
    all_indicators = {}
    
    for symbol in symbols:
        if symbol not in all_data:
            print(f"⚠️ No data available for {symbol}, skipping...")
            continue
            
        print(f"🔧 Processing {symbol}...")
        
        try:
            # Extract OHLC data
            close = prices_df[(symbol, 'close')]
            high = prices_df[(symbol, 'high')]
            low = prices_df[(symbol, 'low')]
            open_price = prices_df[(symbol, 'open')]
            volume = prices_df.get((symbol, 'tick_volume'), pd.Series(index=prices_df.index, data=1000))
        except KeyError:
            print(f"⚠️ Missing OHLC data for {symbol}, skipping...")
            continue
        
        # Initialize indicators DataFrame
        indicators = pd.DataFrame(index=close.index)
        
        # === MOMENTUM INDICATORS ===
        try:
            # RSI variations
            indicators['rsi'] = RSIIndicator(close=close, window=14).rsi()
            indicators['rsi_fast'] = RSIIndicator(close=close, window=7).rsi()
            indicators['rsi_slow'] = RSIIndicator(close=close, window=21).rsi()
            
            # ROC variations
            indicators['roc'] = ROCIndicator(close=close, window=10).roc()
            indicators['roc_fast'] = ROCIndicator(close=close, window=5).roc()
            
            # Stochastic
            stoch = StochasticOscillator(high=high, low=low, close=close)
            indicators['stoch_k'] = stoch.stoch()
            indicators['stoch_d'] = stoch.stoch_signal()
            
            # Additional momentum indicators
            indicators['momentum'] = close.pct_change(10)  # 10-period momentum
            indicators['williams_r'] = ((high.rolling(14).max() - close) / 
                                      (high.rolling(14).max() - low.rolling(14).min())) * -100
            
        except Exception as e:
            logger.warning(f"Some momentum indicators failed for {symbol}: {e}")
        
        # === TREND INDICATORS ===
        try:
            # MACD
            macd = MACD(close=close)
            indicators['macd'] = macd.macd()
            indicators['macd_signal'] = macd.macd_signal()
            indicators['macd_histogram'] = macd.macd_diff()
            
            # ADX
            adx_indicator = ADXIndicator(high=high, low=low, close=close)
            indicators['adx'] = adx_indicator.adx()
            indicators['adx_pos'] = adx_indicator.adx_pos()
            indicators['adx_neg'] = adx_indicator.adx_neg()
            
            # CCI
            indicators['cci'] = CCIIndicator(high=high, low=low, close=close).cci()
            
            # Trend direction
            indicators['trend_direction'] = (close > close.rolling(20).mean()).astype(int)
            
        except Exception as e:
            logger.warning(f"Some trend indicators failed for {symbol}: {e}")
        
        # === VOLATILITY INDICATORS ===
        try:
            # ATR
            atr_indicator = AverageTrueRange(high=high, low=low, close=close)
            indicators['atr'] = atr_indicator.average_true_range()
            indicators['atr_norm'] = indicators['atr'] / close  # Normalized ATR
            
            # Bollinger Bands
            bb = BollingerBands(close=close, window=20, window_dev=2)
            indicators['bb_upper'] = bb.bollinger_hband()
            indicators['bb_lower'] = bb.bollinger_lband()
            indicators['bb_middle'] = bb.bollinger_mavg()
            indicators['bb_width'] = bb.bollinger_wband()
            indicators['bb_position'] = (close - bb.bollinger_lband()) / (bb.bollinger_hband() - bb.bollinger_lband())
            
            # Volatility measures
            indicators['volatility_5'] = close.rolling(5).std()
            indicators['volatility_20'] = close.rolling(20).std()
            indicators['volatility_50'] = close.rolling(50).std()
            
        except Exception as e:
            logger.warning(f"Some volatility indicators failed for {symbol}: {e}")
        
        # === PRICE-BASED FEATURES ===
        try:
            # Returns
            indicators['return_1'] = close.pct_change(1)
            indicators['return_5'] = close.pct_change(5)
            indicators['return_10'] = close.pct_change(10)
            indicators['return_20'] = close.pct_change(20)
            
            # Moving averages
            indicators['sma_5'] = close.rolling(5).mean()
            indicators['sma_10'] = close.rolling(10).mean()
            indicators['sma_20'] = close.rolling(20).mean()
            indicators['sma_50'] = close.rolling(50).mean()
            
            indicators['ema_5'] = close.ewm(span=5).mean()
            indicators['ema_10'] = close.ewm(span=10).mean()
            indicators['ema_20'] = close.ewm(span=20).mean()
            
            # Price position
            indicators['price_position_20'] = (close - close.rolling(20).min()) / (close.rolling(20).max() - close.rolling(20).min())
            indicators['price_position_50'] = (close - close.rolling(50).min()) / (close.rolling(50).max() - close.rolling(50).min())
            
            # Price momentum
            indicators['momentum_slope'] = close.diff(1)
            indicators['price_acceleration'] = indicators['momentum_slope'].diff(1)
            
        except Exception as e:
            logger.warning(f"Some price features failed for {symbol}: {e}")
        
        # === TIME-BASED FEATURES ===
        try:
            indicators['hour'] = indicators.index.hour
            indicators['day_of_week'] = indicators.index.dayofweek
            indicators['month'] = indicators.index.month
            indicators['quarter'] = indicators.index.quarter
            indicators['is_weekend'] = (indicators.index.dayofweek >= 5).astype(int)
            
            # Market session
            def get_market_session(hour):
                if 0 <= hour < 8:
                    return 1  # Asian
                elif 8 <= hour < 16:
                    return 2  # European
                else:
                    return 3  # US
            
            indicators['market_session'] = indicators['hour'].apply(get_market_session)
            
        except Exception as e:
            logger.warning(f"Time features failed for {symbol}: {e}")
        
        # === DATA CLEANING ===
        # Forward fill, backward fill, then fill with appropriate values
        for col in indicators.columns:
            if indicators[col].dtype in ['float64', 'int64']:
                indicators[col] = indicators[col].ffill().bfill()
                
                # Fill remaining NaN with neutral values
                if 'return' in col or 'momentum' in col or 'acceleration' in col:
                    indicators[col] = indicators[col].fillna(0)
                elif any(x in col for x in ['rsi', 'williams_r']):
                    indicators[col] = indicators[col].fillna(50)
                elif 'position' in col:
                    indicators[col] = indicators[col].fillna(0.5)
                else:
                    indicators[col] = indicators[col].fillna(0)
        
        # Replace infinite values
        indicators = indicators.replace([np.inf, -np.inf], np.nan)
        indicators = indicators.ffill().bfill().fillna(0)
        
        all_indicators[symbol] = indicators
        print(f"  ✅ {symbol}: {indicators.shape[0]} rows, {indicators.shape[1]} features")
    
    # Cache the results
    if use_cache:
        data_loader.cached_indicators = all_indicators
    
    print(f"\n🎉 Enhanced indicators calculated for {len(all_indicators)} symbols")
    return all_indicators

# Calculate indicators for all symbols
all_indicators = calculate_enhanced_indicators(multi_prices, SYMBOLS)

print(f"\n📊 Technical Indicators Summary:")
for symbol in SYMBOLS:
    if symbol in all_indicators:
        indicator_df = all_indicators[symbol]
        print(f"  {symbol}: {indicator_df.shape[0]} rows × {indicator_df.shape[1]} features")
    else:
        print(f"  {symbol}: ❌ No indicators calculated")

## 5. Multi-Symbol Optimization Framework

## 5. Multi-Symbol Optimization Framework

The complete MultiSymbolOptimizer implementation is in cell 5 above. This framework includes:

- Enhanced hyperparameter space with symbol-specific adjustments
- Advanced model architectures with flexible configurations  
- Multi-objective optimization combining accuracy and Sharpe ratio
- Comprehensive result tracking and benchmarking
- Cross-validation with time series splits
- Feature selection and data preparation utilities

The optimizer supports:
- Single symbol optimization with study resumption
- Parameter transfer learning between symbols
- Intelligent symbol prioritization
- Advanced model architectures (CNN-LSTM)
- Real-time progress tracking